http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_digits.html

In [ ]:
"""
============================================================
Parameter estimation using grid search with cross-validation
============================================================

This examples shows how a classifier is optimized by cross-validation,
which is done using the :class:`sklearn.grid_search.GridSearchCV` object
on a development set that comprises only half of the available labeled data.

The performance of the selected hyper-parameters and trained model is
then measured on a dedicated evaluation set that was not used during
the model selection step.

More details on tools available for model selection can be found in the
sections on :ref:`cross_validation` and :ref:`grid_search`.

"""

In [2]:
from sklearn import datasets
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from pandas import DataFrame as DF
import tak.tak as tw
from tak.tak import myprint
# Loading the Digits dataset
digits = datasets.load_digits()

In [8]:
print(type(digits))
myprint(digits.keys())
myprint(digits.images.shape)

<class 'sklearn.datasets.base.Bunch'>
digits.keys() = ['images', 'data', 'target_names', 'DESCR', 'target']
digits.images.shape = (1797, 8, 8)


In [10]:
# To apply an classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
X = digits.images.reshape((n_samples, -1))
y = digits.target

myprint(X.shape)

X.shape = (1797, 64)


In [11]:
# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=0)

myprint(X_train.shape)
myprint(X_test.shape)
myprint(y_train.shape)
myprint(y_test.shape)

X_train.shape = (898, 64)
X_test.shape = (899, 64)
y_train.shape = (898,)
y_test.shape = (899,)


In [13]:
# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']
DF(tuned_parameters)

,C,gamma,kernel
0,"[1, 10, 100, 1000]","[0.001, 0.0001]",[rbf]
1,"[1, 10, 100, 1000]",NaN,[linear]


In [14]:
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=5,
                       scoring='%s_weighted' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Note the problem is too easy: the hyperparameter plateau is too flat and the
# output model is the same for precision and recall with ties in quality.

# Tuning hyper-parameters for precision
()
Best parameters set found on development set:
()
{'kernel': 'rbf', 'C': 10, 'gamma': 0.001}
()
Grid scores on development set:
()
0.987 (+/-0.018) for {'kernel': 'rbf', 'C': 1, 'gamma': 0.001}
0.959 (+/-0.030) for {'kernel': 'rbf', 'C': 1, 'gamma': 0.0001}
0.988 (+/-0.018) for {'kernel': 'rbf', 'C': 10, 'gamma': 0.001}
0.982 (+/-0.027) for {'kernel': 'rbf', 'C': 10, 'gamma': 0.0001}
0.988 (+/-0.018) for {'kernel': 'rbf', 'C': 100, 'gamma': 0.001}
0.982 (+/-0.026) for {'kernel': 'rbf', 'C': 100, 'gamma': 0.0001}
0.988 (+/-0.018) for {'kernel': 'rbf', 'C': 1000, 'gamma': 0.001}
0.982 (+/-0.026) for {'kernel': 'rbf', 'C': 1000, 'gamma': 0.0001}
0.974 (+/-0.014) for {'kernel': 'linear', 'C': 1}
0.974 (+/-0.014) for {'kernel': 'linear', 'C': 10}
0.974 (+/-0.014) for {'kernel': 'linear', 'C': 100}
0.974 (+/-0.014) for {'kernel': 'linear', 'C': 1000}
()
Detailed classification report:
()
The model is trained on the full development set.
The scores ar